# Install libraly

In [1]:
! pip install google-cloud-bigquery-storage
! pip3 install pyarrow
! pip3 install fastavro

In [2]:
!pip show google-cloud-bigquery-storage


# for dependens
! pip3 freeze > requirements.txt

Name: google-cloud-bigquery-storage
Version: 0.7.0
Summary: BigQuery Storage API API client library
Home-page: https://github.com/GoogleCloudPlatform/google-cloud-python
Author: Google LLC
Author-email: googleapis-packages@google.com
License: Apache 2.0
Location: /usr/local/lib/python3.6/dist-packages
Requires: google-api-core
Required-by: 


In [0]:
import pandas as pd
import pandas_profiling
import numpy as np
from google.oauth2 import service_account
from google.cloud import *
# import pyarrow as pa



In [0]:
# logging need for check loading progress
import logging
logger = logging.getLogger('pandas_gbq')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [0]:
# initial project
project_id='bionic-tracer-254818'

In [0]:
# get credentianls and Authenticating with a Service Account
credentials = service_account.Credentials.from_service_account_file(
    'bionic-tracer-254818-38d6dd6ee2db.json')

In [0]:
query_tst = """
SELECT *
FROM `bigquery-public-data.iowa_liquor_sales.sales` 
limit 1000000
"""

In [9]:
df_iq_retail = pd.read_gbq(query_tst, 
                 project_id=project_id, 
                 credentials=credentials,      
                  configuration = {'query': {'allowLargeResult': True}},
                  dialect='standard')

Requesting query... 
Query running...
Job ID: 15b6e013-a9a4-4ca9-86dc-277b50cf5363
  Elapsed 6.29 s. Waiting...
  Elapsed 7.33 s. Waiting...
  Elapsed 8.41 s. Waiting...
  Elapsed 9.51 s. Waiting...
  Elapsed 10.63 s. Waiting...
  Elapsed 11.8 s. Waiting...
  Elapsed 12.9 s. Waiting...
  Elapsed 14.02 s. Waiting...
  Elapsed 15.11 s. Waiting...
  Elapsed 16.25 s. Waiting...
  Elapsed 17.36 s. Waiting...
Query done.
Processed: 4.6 GB Billed: 4.6 GB
Standard price: $0.02 USD

Got 1000000 rows.

Total time taken 267.7 s.
Finished at 2020-01-25 19:02:24.


In [10]:
df_iq_retail.sample(5)

,invoice_and_item_number,date,store_number,store_name,address,city,zip_code,store_location,county_number,county,category,category_name,vendor_number,vendor_name,item_number,item_description,pack,bottle_volume_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale_dollars,volume_sold_liters,volume_sold_gallons
56129,INV-00160800031,2016-09-06,3993,New Star Liquor / W 4th S / Waterloo,1625 West 4th St,Waterloo,50701,None,7,BLACK HAWK,None,None,115,CONSTELLATION BRANDS INC,53210,Paul Masson Grande Amber Brandy,48,100,1.00,1.50,48,72.00,4.8,1.27
82465,INV-12122900026,2018-05-16,4197,Fareway Stores #014 / Sergeant Bluff,700 1st St,Sergeant Bluff,51054,POINT (-96.353078 42.400504),97,WOODBURY,1062100,Gold Rum,434,LUXCO INC,45248,Paramount Gold Rum,6,1750,7.84,11.76,6,70.56,10.5,2.77
327763,S07362000087,2012-08-28,2522,Hy-Vee Wine and Spirits / Spirit Lak,HWY 9 & 71,SPIRIT LAKE,51360,None,30,Dickinson,1081030,COFFEE LIQUEURS,65,Jim Beam Brands,67556,Kamora Coffee Liqueur,12,750,6.29,9.44,12,113.28,9.0,2.38
928197,S14054500037,2013-08-22,4677,Costco Wholesale #1111 / Coralville,2900 HEARTLAND DR,CORALVILLE,52241,POINT (-91.61494100000002 41.698028),52,Johnson,1701100,DECANTERS & SPECIALTY PACKAGES,260,Diageo Americas,3887,Bailey's Original w/Ice Cream Bowl,6,1000,17.25,25.88,30,776.40,30.0,7.93
142995,INV-10490700001,2018-02-20,3625,Wal-Mart 0892 / Ankeny,1002 SE National Dr,Ankeny,50021,POINT (-93.582348 41.70486400000001),77,POLK,1081500,Triple Sec,065,Jim Beam Brands,86112,Dekuyper Triple Sec,12,1000,2.33,3.50,12,42.00,12.0,3.17


# Generate report for data set use libraly 
[*pandas-profiling*](https://github.com/pandas-profiling/pandas-profiling) 


---
# part I - Overview


In [11]:
pandas_profiling.ProfileReport(df_iq_retail)

/usr/local/lib/python3.6/dist-packages/pandas/plotting/_matplotlib/converter.py:103: FutureWarning: Using an implicitly registered datetime converter for a matplotlib plotting method. The converter was registered by pandas on import. Future versions of pandas will require you to explicitly register matplotlib converters.

To register the converters:
	>>> from pandas.plotting import register_matplotlib_converters
	>>> register_matplotlib_converters()
  warnings.warn(msg, FutureWarning)
/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,24
Number of observations,1000000
Total Missing (%),0.6%
Total size in memory,183.1 MiB
Average record size in memory,192.0 B
Numeric,5
Categorical,14
Boolean,0
Date,1
Text (Unique),1
Rejected,3
